In [1]:
from requests_html import HTMLSession
import pandas as pd
from time import sleep
import requests

# keep track of loading progress
from tqdm.notebook import tqdm

import pathlib
import time

In [2]:
def get_ticker(url, session):
    r = session.get(url)
    content = r.html.find('section.release-body')
    try:
        for item in content:
            ticker = item.find('a.ticket-symbol', first=True).text
    except AttributeError:
        ticker = None
    try:
        return ticker
    except UnboundLocalError:
        return None # Return non if no tickers found

In [3]:
def get_page_details(x, session,data=[]):
    url = f'https://www.prnewswire.com/news-releases/news-releases-list/?page={x}&pagesize=100'
    r = session.get(url)
    content = r.html.find('div.row.arabiclistingcards')
    for item in tqdm(content, desc='Parsing page...\t', leave=False):
        date = item.find('h3', first=True).text.split('ET')[-2]
        title = item.find('h3', first=True).text.split('ET')[-1]
        article_url = 'https://www.prnewswire.com' + item.find('a.newsreleaseconsolidatelink', first=True).attrs['href']
        ticker = get_ticker(article_url, session)
        try:
            dic = {
              'Date': pd.to_datetime(date),
              'Title': title,
              'Ticker': ticker,
              'Article URL': article_url
            }
            data.append(dic)
        except Exception:
            pass
        
    return data

In [4]:
def get_news(pages):
    session = HTMLSession()
    data = []

    for x in tqdm(range(1, pages+1), desc='Loading Pages...\t'):
        get_page_details(x, session,data)
    
    df = pd.DataFrame(data)
    return df

In [5]:
df = get_news(2)

In [6]:
print(df)

                   Date                                              Title  \
0   2021-11-21 21:27:00   Ding Lei: New Ultra-Futuristic HiPhi Z from H...   
1   2021-11-21 21:15:00   Pace raises USD40 million Series A funding fr...   
2   2021-11-21 20:53:00            Lanvin Group Unveils New Brand Identity   
3   2021-11-21 20:41:00   Aston Martin Vantage mit BAPE+GEEKVAPE-Beschi...   
4   2021-11-21 20:34:00   New Models, New Energy | GAC at the Auto Guan...   
..                  ...                                                ...   
194 2021-11-19 17:42:00   Lucid Named To Inaugural Inc. Best-Led Compan...   
195 2021-11-19 17:41:00   Nabors Energy Transition Corp. Announces Clos...   
196 2021-11-19 17:39:00   Cyber Defense Labs' CEO Robert Anderson Jr, N...   
197 2021-11-19 17:30:00   Agnico Eagle and Kirkland Lake Gold Remind Sh...   
198 2021-11-19 17:30:00   Water Bottles with Filters Market Size to inc...   

    Ticker                                        Article URL  

In [7]:
df.to_csv('Outputfile.csv', index=False)